# [SOLUTION] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [4]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [5]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_api_key

'voc-2015200036168865441126268e0ed5be47456.24871461'

## Play with Tavily

In [6]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [7]:
result = client.search("What's Nintendo?")

In [8]:
result

{'query': "What's Nintendo?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Nintendo',
   'title': 'Nintendo - Wikipedia',
   'content': '34°58′11″N 135°45′22.3″E\ufeff / \ufeff34.96972°N 135.756194°E\ufeff / 34.96972; 135.756194 **Nintendo Co., Ltd.** is a Japanese multinational video game company headquartered in Kyoto. The history of Nintendo began when craftsman Fusajiro Yamauchi founded the company to produce handmade *hanafuda* playing cards. The company became internationally dominant in the 1980s after the arcade release of *Donkey Kong "Donkey Kong (1981 video game)")* (1981) and the Nintendo Entertainment System, which launched outside of Japan alongside *Super Mario Bros.* in 1985. Nintendo was founded as Nintendo Koppai on 23 September 1889 by craftsman Fusajiro Yamauchi in Shimogyō-ku, Kyoto, Japan, as an unincorporated establishment, to produce and distribute Japanese playing cards, or karuta (かるた; from P

## Define Web Search tool

In [9]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

In [10]:
tools = [web_search]

In [11]:
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=("You are a helpful assistant"),
)

In [12]:
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
            "You are a web-aware assistant that can search for update information "
            "For each query, you will search the web for current information using " 
            "Tavily's AI-optimized search and provide a comprehensive answer\n"
            "Always cite your sources and explain any discrepancies found.\n"
            "Be particularly attentive to dates and time-sensitive information."
    ),
    tools=tools
)

In [13]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [14]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: voc-2015**************************************1461. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information on events or outcomes beyond October 2023, so I cannot provide details about the 2025 Oscars or the winner of the International Movie category. You may want to check the latest news sources or the official Oscars website for the most up-to-date information.


In [ ]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events or outcomes beyond October 2023, so I cannot provide details about the 2025 Oscars or the winner of the International Movie category. You may want to check the latest news sources or the official Oscars website for the most up-to-date information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2023, several key developments in AI technology were notable:

1. **Generative AI Advancements**: Generati

In [ ]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2023, several key developments in AI technology were notable:

1. **Generative AI Advancements**: Generative models, especially in natural language processing (NLP) and computer vision, continued to evolve. Models like GPT-4 and its successors demonstrated improvements in understanding and generating human-like text, with applications in creative writing, coding, and customer service.

2. **AI in Healthcare**: AI applications in healthcare saw significant progress, including improved diagnostic tools using machine learning for early detection of diseases like cancer and advancements in personalized medicine through genetic data analysis.

3. **AI Ethics and Regulation**: There was an increasing focus on ethical AI and the need for regulations to govern AI development and usage. Discussions around transparency, bias, and the societal impact of AI technologies gained prominence, with various governments and organizations working on frameworks for

**Web Agent**

In [ ]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_qzAPaG8fyOwkDFkY6ZdJlYkq', function=Function(arguments='{"query":"2025 Oscar winner for Best International Feature Film","search_depth":"advanced"}', 

In [ ]:
print(run1.get_final_state()["messages"][-1].content)

The 2025 Oscar for Best International Feature Film was awarded to **"I'm Still Here,"** a film from Brazil. This win marked Brazil's first Oscar in this category. The award was presented during the 97th Academy Awards held on March 2, 2025. 

Other nominees for this category included:
- "The Girl with the Needle" (Denmark)
- "Emilia Pérez" (France)
- "The Seed of the Sacred Fig" (Germany)
- "Flow" (Latvia)

"I'm Still Here" is based on a true story set in 1970s Rio de Janeiro during Brazil's military dictatorship, showcasing the struggle for freedom and identity during this tumultuous period ([The Hollywood Reporter](https://www.hollywoodreporter.com/lists/oscars-2025-winners-list/), [NPR](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature), [People](https://people.com/oscars-2025-im-still-here-brazil-best-international-feature-11684447)).


In [ ]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_qzAPaG8fyOwkDFkY6ZdJlYkq', function=Function(arguments='{"query":"2025 Oscar winner for Best International Feature Film","search_depth":"advanced"}', 

In [ ]:
print(run2.get_final_state()["messages"][-1].content)

As of 2025, significant advancements in AI technology have been reported, highlighting key developments from major tech companies:

1. **Google's Gemini 2.0**: Released in March 2025, Gemini 2.0 is Google's most advanced AI model to date. It features agentic capabilities, enhancing its usability for developers, enterprises, and individuals. This model is part of Google's ongoing efforts to push the boundaries of AI technology across different applications.

2. **New AI Models**: Google also introduced Veo 2 and Imagen 3, which are advanced models for video and image generation, respectively. These models have achieved state-of-the-art results in comparative evaluations, showcasing Google's commitment to excellence in AI.

3. **Alibaba's Qwen3**: In May 2025, Alibaba launched Qwen3, which has significantly narrowed the technological gap with leading U.S. AI firms. Qwen3 boasts advancements in cost efficiency and multilingual capabilities, positioning it as a viable alternative for globa

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow